# Historical Predictions
This notebook contains the code for the Historical Predictions.

In [ ]:
cd ..

In [ ]:
from lib import estimator_util as eu
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import string

In [ ]:
sns.set_theme()

In [ ]:
complete_df, all_cols = eu.get_data("../data/lsms/processed/_all_nominal.csv", "../data/cnn_features/resnet_trans_all_countries_hyper.csv", "../data/")

## All Countries
Historical future and past predictions for all countries.

In [ ]:
def all_year_run(complete_df, model, **params):
    countries = complete_df.groupby(["country"]).groups.keys()
    base_year = []
    year_ = []
    r2 = []
    country_ = []
    for country in countries:
        years = complete_df.loc[complete_df["country"] == country].groupby(["year"]).groups.keys()
        for year in years:
            for x_year in years:
                X_base, y_base = eu.get_features(complete_df, [country], [year], all_cols)
                if year == x_year:
                    r, _,_, m = eu.run_model(X_base, y_base, model, **params)
                    country_.append(country)
                    base_year.append(year)
                    year_.append(x_year)
                    r2.append(r)
                    continue
                infl = eu.get_inflation_perf(country, year, x_year)
                X, y = eu.get_features(complete_df, [country], [x_year], all_cols, infl=infl, pca_comp_cnn=5)
                n = len(X_base)
                X = StandardScaler().fit_transform(X)
                r, y_predicted, m = eu.run_model_out(X_base, y_base, X, y, model, **params)
                country_.append(country)
                base_year.append(year)
                year_.append(x_year)
                r2.append(r)
    return pd.DataFrame.from_dict({"Country": country_,"Train Year": base_year, "Eval Year": year_, "r2": r2})
        

### Prediction with Ridge

In [ ]:
all_year_run(complete_df, Ridge, alpha = 1000)

### Prediction with CatBoost

In [ ]:
all_year_run(complete_df, CatBoostRegressor, random_state=42)

## Leave-one-out

In [ ]:
def leave_one_out_time_prediction(df, osm_cols, model, **params) :
    countries = ["ETH", "NG"]
    for country in countries:
        print(f"======{country}=====")
        years = list(df.loc[df["country"] == country].groupby(["year"]).groups.keys())
        print(years)
        loo = LeaveOneOut()
        X_ = None
        y_ = None
        for year, x_year in loo.split(years):
            for y in year:
                infl = eu.get_inflation_perf(country, 2010, years[y])
                X_base, y_base = eu.get_features(df, [country], [years[y]], osm_cols, infl)
                
                if X_ is None:
                    X_ = X_base
                else:
                    X_ = np.vstack((X_, X_base))
                
                if y_ is None:
                    y_ = y_base
                else:
                    y_ = np.append(y_, y_base)

            infl = eu.get_inflation_perf(country, 2010, years[x_year[0]])
            X, y = eu.get_features(df, [country], [years[x_year[0]]], osm_cols, infl=infl)
            X_ = StandardScaler().fit_transform(X_)
            X = StandardScaler().fit_transform(X)
            r, y_hest, m = eu.run_model_out(X_, y_, X, y, model, **params)
            print(f"Predicting: {years[x_year[0]]}")
            print(r)

### with Ridge for comparison

In [ ]:
leave_one_out_time_prediction(complete_df, all_cols, Ridge, alpha=1000)

### with CatBoost for best model

In [ ]:
leave_one_out_time_prediction(complete_df, all_cols, CatBoostRegressor, task_type="GPU", random_state=42)

## Pooled Dataset
Predictions for whole dataset. Consumption is scaled to base year 2010.

In [ ]:
def combined_time_travel(df, osm_cols, model, **params) :
    countries = [["NG"], ["ETH"] ,["TZA"], ["MW"], ["NG", "ETH", "TZA", "MW"]]
    results = np.zeros((5,5))
    for i, country in enumerate(countries):
        X, y = eu.get_features_allyears(df, country, osm_cols)
        for j, out_country in enumerate(countries):
            X_out, y_out = eu.get_features_allyears(df,out_country, osm_cols)
            r, _, _= eu.run_model_out(X, y, X_out, y_out, model, **params)
            results[i,j] = r

    countries = ["Nigeria", "Ethiopia" ,"Tanzania", "Malawi", "Pooled"]
    df_cm = pd.DataFrame(results, index = countries, columns = countries)
    fig = plt.figure(figsize = (10,7))
    sns.heatmap(df_cm, annot=True, cbar_kws={'label': r'$r^2$'})
    plt.xlabel("Country trained on")
    plt.ylabel("Country evaluated on")
    plt.title("Heatmap of the cross country predictions with " + model.__name__)
    ax = plt.gca()
    plt.text(-0.1, 1.1, string.ascii_uppercase[0], size=20, weight='bold', transform=ax.transAxes)
    return fig

In [ ]:
fig = combined_time_travel(complete_df, all_cols, Ridge, alpha=1000)
fig.savefig("../figs/cross_country_pooled_timetravel_ridge.pdf", dpi=600, bbox_inches="tight")

### with CatBoost

In [ ]:
fig = combined_time_travel(complete_df, all_cols, CatBoostRegressor, task_type="GPU", random_state=42)
fig.savefig("../figs/cross_country_pooled_timetravel_catboost.pdf", dpi=600, bbox_inches="tight")

# Plot pooled combined

In [ ]:
def plot_pooled_combined_time_travel(complete_df, all_cols, model, **params) :
    country = ["NG", "ETH", "TZA", "MW"]
    X, y = eu.get_features_allyears(complete_df, country, all_cols)
    r, _, _, m = eu.run_model(X, y, model, **params)
    y_pred = m.predict(X)
    eu.plot_predictions(y, y_pred, r, "Pooled", "Timetravel")
    

### with Ridge

In [ ]:
plot_pooled_combined_time_travel(complete_df, all_cols, Ridge, alpha = 1000)

### with CatBoost

In [ ]:
plot_pooled_combined_time_travel(complete_df, all_cols, CatBoostRegressor, task_type="GPU", random_state=42)